# Download all twitter politicians

The following webpage has all the danish politician twitterprofiles  https://filip.journet.sdu.dk/twitter/politikere/

In [ ]:
import requests
from bs4 import BeautifulSoup
import re #https://developers.google.com/edu/python/regular-expressions
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm

In [ ]:
from tqdm.notebook import tqdm
URL = 'https://filip.journet.sdu.dk/twitter/politikere/'
page = requests.get(URL)

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')


## Selecting the twitternames

See that the twitter names is in a h3 style:

In [ ]:
twitter_names = soup.find_all('h3')

In [ ]:
#twitter_names

In [ ]:
twitter_name_list=[]

In [ ]:
for names in twitter_names:
   # print(names, end='\n'*2)
    match=re.search("<h3>#",str(names))
    if match:
        result_name=re.findall(".com/\w*",str(names))[0].replace(".com/","")
        twitter_name_list.append(result_name.rstrip())
        print(result_name)

## The real names

The real names is in a < small > class.

In [ ]:
real_name_list=[]

In [ ]:
real_names = soup.find_all('small')

In [ ]:
for r_names in real_names:
    result=re.search(r'<small>.+',str(r_names)).group(0).replace("<small>","")
    real_name_list.append(result.rstrip())
    print(result)

In [ ]:
len(real_name_list)

## Making dataframe

In [ ]:
d={'Names':real_name_list,"Twittername":twitter_name_list}
data=pd.DataFrame(data=d)

In [ ]:
data.to_csv("danish_politkere_twitter.csv")

In [ ]:
data.head()

## Finding policial parties from https://www.danskepolitikere.dk/

In [ ]:
# load csv
data = pd.read_csv("danish_politkere_twitter.csv")
data = data.drop(columns=['Unnamed: 0'])
soup = BeautifulSoup(requests.get('https://www.danskepolitikere.dk/oversigt/danske-politikere').content, 'html.parser')

In [ ]:
sections = list(soup.find_all('section'))

In [ ]:
# add "UNKNOWN" as default party
data['Party'] = 'UNKNOWN'
data['Region'] = 'UNKNOWN'
data.head()

In [ ]:
parties = []
# cross reference for our politicians and check if he's in the party
for row, coloumns in data.iterrows(): 
    name = coloumns[0] 
    for section in sections:
        party = section.h3.a.text # gets the party 
        if re.search(name.lower(), str(section).lower()):
            data.at[row, 'Party'] = party
            print("Found " + name + " in " + party)
            
        


In [ ]:
# view results
data.groupby('Party').count()

In [ ]:
# get snapshot of who's missing
data.loc[data['Party'] == "UNKNOWN"]


Too many still unkown so we must look elsewhere, in denmark we have many
regions with politicans that aren't necsarily members of the parlament
![Regioner i dk](https://rn.dk/-/media/Rn_dk/Genveje/Fakta-om-Nordjylland/Regioner-i-Danmark/Danmark_Regioner_2017.ashx?la=da)
So we will scrape these websites as well


### first north jutland

In [ ]:
nordjylland_url = "https://rn.dk/da/Politik/De-regionale-politikere"

soup = BeautifulSoup(requests.get(nordjylland_url).content, 'html.parser')



In [ ]:
# examine html
#print(soup.prettify())
match_rnj = []
pre_re = r"\b[A-Z].*"
div_lsit = soup.findAll("ul", {'class': 'span3'})
for paragraph in div_lsit:
    text= paragraph.text
    text_stripped = text.rstrip().lstrip()
    #print(text_stripped)
    matches_1 = re.findall(pre_re, text_stripped)
    print(matches_1)
    match_rnj.append(matches_1)

# 0 and even indices are names, 1 and odd are the matching parti strings

In [ ]:
# note that relevant matches come at last 2 indices
good_match = match_rnj[len(match_rnj)-2] + match_rnj[len(match_rnj)-1]

print(good_match)
# remove strings that contain "formand" as it's not relevant information
good_match = [x for x in good_match if "formand" not in x.lower() if "\r" not in x.lower()]
print(good_match)
print(len(good_match))

for i in range(len(good_match)-1):
    name = good_match[i]
    party_string = good_match[i+1]
    if name in data.values:
        print(name+ "found in "+ party_string.replace("Parti: ", ""))
        data.loc[data['Names'] == name, 'Party'] = party_string.replace("Parti: ", "")
        data.loc[data['Names'] == name, 'Region'] = 'nordjylland'


In [ ]:
data.groupby('Party').count()

# Region Midtjylland

In [ ]:
url = 'https://www.rm.dk/politik/regionsradsmedlemmer/'

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

content = soup.find("div", {"class": "text-content"})

unwanted = content.findAll("p")
for u in unwanted:
    u.clear()

# removing some introductioniary text
unwanted = content.find("h2")
unwanted.clear()
unwanted.find_next().clear()
clean_content = content.text.rstrip()

In [ ]:
# removing unwanted brackets ()
pattern = r"\(.*?\)"
clean_content = re.sub(pattern, '', clean_content)


#print(clean_content) # left with this string

In [ ]:
all_parties = content.findAll("h3")
for party in all_parties:
    #print(party.text)
    party_text = re.sub(pattern,'', party.text)
    people = party.find_next('ul')
    people_text = re.sub(pattern, "", people.text)
    for name in people_text.split('\n'):
        if name in data.values:
            print(name + "  in   " + party_text)
            data.loc[data['Names'] == name, 'Party'] = party_text.rstrip()
            data.loc[data['Names'] == name, 'Region'] = 'midtjylland'

In [ ]:
data.groupby('Party').count()

In [ ]:
data.Party.unique()

# Region syddammark

In [ ]:
url = 'https://www.regionsyddanmark.dk/wm436558'
req = requests.get(url)

soup = BeautifulSoup(req.content, 'html.parser')
content = soup.find("div", {"class": "brodtekst"})

unwanted = content.findAll("p")
for u in unwanted:
    u.clear()


In [ ]:
all_parties = content.findAll("h3")
i = 0
for party in all_parties:
        party_text = re.sub(pattern,'', party.text)
        people = party.find_next('ul')
        people_text = re.sub(pattern, "", people.text)
        for people in people_text.split('\n'):
            if people.strip() in data.values:
                print(people.strip() + "  in   " + party_text.strip())
                data.loc[data['Names'] == people.strip(), 'Party'] = party_text.strip()
                data.loc[data['Names'] == name, 'Region'] = 'syddanmark'

In [ ]:
data.groupby('Party').count()

# Region Sjælland

In [ ]:
url = 'https://www.regionsjaelland.dk/politik/regionsraadet/regionsraadets-medlemmer/Sider/default.aspx'
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

content = soup.findAll('div',{'class': 'employeList'})
content = content[1:]
for name_content in content:
    name = name_content.a['title']
    spans = name_content.findAll('span')
    party = spans[1]

    party_text = party.text.split(',')[1]
    party_text = re.sub(pattern, '', party_text)
    if name.strip() in data.values:
        print(name.strip() + "  in   " + party_text.strip())
        data.loc[data['Names'] == name.strip(), 'Party'] = party_text.strip()
        data.loc[data['Names'] == name, 'Region'] = 'sjælland'


In [ ]:
data.groupby('Party').count()

# Region Hovedstaden

In [ ]:
url = 'https://www.regionh.dk/politik/politiske-udvalg-og-fora/regionsraadet/Sider/Medlemmer_af_regionsraadet_2018-2021.aspx'

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
content = soup.findAll('h2', {'class': 'rh-Element-H2A'})

for party in content:
    print(party.text)
    party_text = re.sub(pattern, '', party.text)
    print(party_text.strip())
    print("")
    names = party.find_all_next('h3', {'rh-Element-H3A'})
    # bit weird website structure compared to others so differnet method is used
    # each party returns the names of politicians + those listed after it
    # we can update each name setting it to the party continously for each element
    for name in names:
        name_text = name.text
        name_text = name_text.strip()
        #print(name_text)
        if name_text in data.values:
                print(name_text + " in " + party_text.strip())
                data.loc[data['Names'] == name_text, 'Party'] = party_text.strip()
                data.loc[data['Names'] == name, 'Region'] = 'hovedstaden'


In [ ]:
data.groupby('Party').count()

In [ ]:
# get snapshot of who's missing
data.loc[data['Party'] == "UNKNOWN"]

## Finally from ft.dk

In [ ]:
### nordjyllands storkreds
def getFromFt(url: str, region: str):
    req=requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    memberInfos = soup.findAll('tr')
    # skip table header
    memberInfos = memberInfos[1:]
    print("Found " + str(len(memberInfos)))
    for member in memberInfos:
        # format: '\n\n -> first_name -> last_name -> party
        dataList = [x.text for x in member.findAll('td')]
        try:
            _, firstName, lastName, party, _, _ = dataList
            name = firstName + " " + lastName
            if name in data.values:
                print("In twitter csv: " + name + " " + party)
                print("Old party: " + data.loc[data['Names']==name]['Party'])
                data.loc[data['Names'] == name, 'Party'] = party
                data.loc[data['Names'] == name, 'Region'] = region

        except:
            print("failed for")
            print(dataList)



url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti=%7bD68D9A59-8A4B-426D-AD9C-9B32FE3CE071%7d&page=1&sortedBy=&pageSize=200'
getFromFt(url, 'nordjylland')
data.groupby('Party').count()

In [ ]:
## Vestjyllandsstokkreds + øst
# trying same code

url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={97A50F7E-B9D0-4D91-924F-AB20235DB4FD}'
getFromFt(url, 'midtjylland')
data.groupby('Party').count()

# østjylland
url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={E3910753-B21F-4E97-AD2B-67A9A3C15B62}'
getFromFt(url, 'midtjylland')
data.groupby('Party').count()

In [ ]:
# syddanmark
url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={7D7567F4-3425-42A1-A035-2FB9880A5F81}'
getFromFt(url, 'syddanmark')
data.groupby('Party').count()


url ='https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={B3A26626-55E1-4BE2-8503-E4027904E25A}'
getFromFt(url, 'syddanmark')
data.groupby('Party').count()

In [ ]:
url = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti=%7b79B9CCF4-4793-4835-8E68-6770AA11A85D%7d&page=1&sortedBy=&pageSize=50'
getFromFt(url, 'sjælland')
data.groupby('Party').count()

In [ ]:
# Region hovedstadan
url1 = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={4C1C732F-4F3C-408E-A009-C96E6934B29C}'
url2 = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={3F642D1B-1820-48F1-A288-AA1FE9079640}'
url3 = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={6A4F6A85-C5C1-4656-BD9A-7831094A5A81}'
url4 = 'https://www.ft.dk/searchResults.aspx?sortedDescending=false&consti={E76B8CF6-E8A7-4B29-9813-350C49E36238}'

getFromFt(url1, 'hovedstadan')
getFromFt(url2, 'hovedstadan')
getFromFt(url3, 'hovedstadan')
getFromFt(url4, 'hovedstadan')

data.groupby('Party').count()

### Analyzing why there are still so many unknowns

In [ ]:
# unknowns = data.loc[data['Party'] == "UNKNOWN"]
#
# for row in unknowns['Names']:
#     print(row)

Looks like there are several reasons, some names have emojis. Some have misplaced whitespaces, sometimes more
sometimes 0. There are some names in the list that are entities and not people - such as AMPJ or various ministeries.
There can also be trouble finding politicians because of the capitalization (easy fix) and laslty,
some middle names are abbreviated and some aren't; making it harder to know and find the specific politician.


In [ ]:
# write to csv
data.to_csv('danish_politkere_twitter2.csv')
